In [17]:
import sys
import os
import shutil
import cv2
import numpy as np
from pathlib import Path
from typing import List, Tuple, Optional
from tqdm import tqdm

paths_to_add = [
    '/root',         
    '/root/task'   
]
for p in paths_to_add:
    if p not in sys.path:
        sys.path.append(p)

from src.YoloSAM.utils.config import YOLOConfig
from src.YoloSAM.scripts.train_yolo import YOLOTrainer

class MaskToYOLOConverter:
    """Convert mask images to YOLO format labels."""
    
    def __init__(self, class_id: int = 0):
        self.class_id = class_id
    
    def mask_to_bboxes(self, mask: np.ndarray, min_area: int = 20) -> List[Tuple[float, float, float, float]]:
        # 1. 确保二值化
        if mask.max() > 1:
            mask = (mask > 127).astype(np.uint8)
        else:
            mask = mask.astype(np.uint8)
            
        bboxes = []
        height, width = mask.shape

        grid_rows = 8
        grid_cols = 8
        
        step_h = height // grid_rows
        step_w = width // grid_cols
        
        for r in range(grid_rows):
            for c in range(grid_cols):
                # 计算当前网格的坐标
                y_start = r * step_h
                x_start = c * step_w
                # 处理边界情况
                y_end = (r + 1) * step_h if r < grid_rows - 1 else height
                x_end = (c + 1) * step_w if c < grid_cols - 1 else width
                
                # 取出当前网格的 Mask 碎片
                patch = mask[y_start:y_end, x_start:x_end]
                
                # 如果这个格子里没有血管，跳过
                if patch.sum() == 0:
                    continue
                
                # 在这个小格子里找轮廓
                contours, _ = cv2.findContours(patch, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                for contour in contours:
                    lx, ly, lw, lh = cv2.boundingRect(contour)
                    
                    # 过滤极小的噪点
                    if lw * lh < min_area:
                        continue
                    
                    # === 坐标还原 ===
                    # 轮廓坐标是相对于 patch 的，要加回 global 坐标
                    global_x = x_start + lx
                    global_y = y_start + ly
                    
                    # 归一化 (YOLO 格式)
                    x_center = (global_x + lw / 2) / width
                    y_center = (global_y + lh / 2) / height
                    width_norm = lw / width
                    height_norm = lh / height
                    
                    bboxes.append((x_center, y_center, width_norm, height_norm))
        
        return bboxes

    
    def convert_single_mask(self, mask_path: Path, output_path: Path, min_area: int = 100):
        mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        if mask is None:
            # Try removing _mask suffix if not found (DRIVE dataset specific)
            return
        
        bboxes = self.mask_to_bboxes(mask, min_area)
        
        output_path.parent.mkdir(parents=True, exist_ok=True)
        with open(output_path, 'w') as f:
            for bbox in bboxes:
                x_center, y_center, width, height = bbox
                f.write(f"{self.class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
    
    def convert_dataset_inplace(self, base_path: str, min_area: int = 100, splits: List[str] = ['train', 'val']):
        base_path = Path(base_path)
        for split in splits:
            print(f"Processing {split} split...")
            split_dir = base_path / split
            mask_dir = split_dir / 'mask' 
            # 兼容性处理：DRIVE 有时候叫 masks 有时候叫 mask
            if not mask_dir.exists(): mask_dir = split_dir / 'masks'
            
            label_dir = split_dir / 'labels'
            
            # === 关键修复：强制清理旧标签 ===
            if label_dir.exists():
                print(f"🧹 Cleaning old labels in {label_dir}...")
                shutil.rmtree(label_dir)
            label_dir.mkdir(parents=True, exist_ok=True)
            
            if not mask_dir.exists():
                print(f"Warning: Mask directory {mask_dir} does not exist")
                continue
            
            # 支持多种格式
            mask_files = list(mask_dir.glob('*.png')) + list(mask_dir.glob('*.jpg')) + list(mask_dir.glob('*.gif'))
            print(f"Found {len(mask_files)} mask files in {split}")
            
            for mask_file in tqdm(mask_files, desc=f"Converting {split}"):
                raw_name = mask_file.stem
                # 修复文件名不匹配问题 (21_training_mask -> 21_training)
                clean_name = raw_name.replace('_mask', '').replace('_manual1', '') 
                label_file = label_dir / f"{clean_name}.txt"
                self.convert_single_mask(mask_file, label_file, min_area)
            
            # === 关键修复：删除 YOLO 缓存 ===
            # 如果不删缓存，YOLO 可能会读取之前“没有标签”的记录
            cache_path = label_dir.parent / 'labels.cache'
            if cache_path.exists():
                os.remove(cache_path)
                print(f"🗑️ Deleted cache: {cache_path}")


converter = MaskToYOLOConverter(class_id=0)
converter.convert_dataset_inplace(
    base_path='../../../datasets/DRIVE',
    min_area=50,  # ⚠️ 调大一点，避免把噪点当成瘢痕，减少框的数量
    splits=['train', 'val']
)

config = YOLOConfig(
    # Model Settings
    model_type="yolo11n",
    device='cuda',
    pretrained_path='../checkpoints',

    # DataSet Settings
    dataset_path='../../../datasets/DRIVE/train',
    val_dataset_path='../../../datasets/DRIVE/val',
    class_names=['scar'],

    epochs=100,
    patience=20,     # 早停耐心值
    batch_size=16,
    image_size=640,

    # Augmentation
    mosaic=0.5,
    degrees=15.0,

    # Detection parameters
    iou_threshold=0.2,
    conf_threshold=0.01, # 训练时可以低一点

    # Project settings
    project_name="yolo_scar_detection",
    experiment_name="final_100epochs",

    max_detections=300, 
    wandb_mode="disabled"
)

trainer = YOLOTrainer(config)
results = trainer.train()

print(f"最佳权重保存在: runs/detect/{config.project_name}/{config.experiment_name}/weights/best.pt")

🚀 开始转换数据...
Processing train split...
🧹 Cleaning old labels in ../../../datasets/DRIVE/train/labels...
Found 100 mask files in train


Converting train: 100%|██████████| 100/100 [00:00<00:00, 237.54it/s]


🗑️ Deleted cache: ../../../datasets/DRIVE/train/labels.cache
Processing val split...
🧹 Cleaning old labels in ../../../datasets/DRIVE/val/labels...
Found 20 mask files in val


Converting val: 100%|██████████| 20/20 [00:00<00:00, 300.71it/s]


🗑️ Deleted cache: ../../../datasets/DRIVE/val/labels.cache
⚙️ 配置训练参数...
🔥 开始训练 YOLO...
Loading pretrained YOLO model: yolo11n
Created dataset configuration at: ../../../datasets/DRIVE/dataset.yaml
Starting YOLO training for 100 epochs
Starting YOLO training with configuration:
  data: ../../../datasets/DRIVE/dataset.yaml
  epochs: 100
  imgsz: 640
  batch: 16
  patience: 20
  box: 7.5
  cls: 0.5
  dfl: 1.5
  iou: 0.2
  max_det: 300
  mosaic: 0.5
  mixup: 0.1
  copy_paste: 0.4
  scale: 0.5
  fliplr: 0.5
  flipud: 0.1
  degrees: 15.0
  translate: 0.3
  hsv_h: 0.0
  hsv_s: 0.0
  hsv_v: 0.3
  lr0: 0.001
  lrf: 0.0001
  warmup_epochs: 10
  weight_decay: 0.001
  save_period: 50
  project: runs
  name: yolo_scar_detection
  multi_scale: True
  device: cuda
Ultralytics 8.3.237 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32109MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, class

In [16]:
from ultralytics import YOLO
import cv2
import os
import matplotlib.pyplot as plt

# 1. 加载你的模型
model_path = "runs/yolo_scar_detection16/weights/best.pt"
print(f"🔥 加载模型: {model_path}")
model = YOLO(model_path)

# 2. 指定图片
img_path = "/root/task/datasets/DRIVE/val/images/01_test.tif"

# 3. 直接预测 (Raw Prediction)
# conf=0.001: 只要有一丁点概率是血管，就给我画出来
# max_det=300: 允许画 300 个框
# iou=0.5: 稍微做点去重
print("🚀 正在进行原生推理 (Conf=0.001)...")
results = model.predict(
    img_path, 
    conf=0.001, 
    iou=0.5, 
    max_det=300, 
    verbose=True
)

# 4. 分析结果
result = results[0]
boxes = result.boxes
print(f"\n📊 统计结果:")
print(f"检测到的框数量: {len(boxes)}")

if len(boxes) > 0:
    print(f"最高置信度: {boxes.conf.max().item():.4f}")
    print(f"最低置信度: {boxes.conf.min().item():.4f}")
    print(f"平均置信度: {boxes.conf.mean().item():.4f}")

# 5. 保存结果图
save_path = "debug_result.jpg"
result.save(filename=save_path)  # Ultralytics 自带的保存功能
print(f"\n✅ 图片已保存至: {os.path.abspath(save_path)}")
print("请去左侧文件浏览器下载/查看这张图片！")

🔥 加载模型: runs/yolo_scar_detection16/weights/best.pt
🚀 正在进行原生推理 (Conf=0.001)...

image 1/1 /root/task/datasets/DRIVE/val/images/01_test.tif: 640x640 300 scars, 7.5ms
Speed: 2.8ms preprocess, 7.5ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

📊 统计结果:
检测到的框数量: 300
最高置信度: 0.0937
最低置信度: 0.0143
平均置信度: 0.0298

✅ 图片已保存至: /root/task/src/YoloSAM/drive/debug_result.jpg
请去左侧文件浏览器下载/查看这张图片！


In [3]:
import sys
import os
import shutil
import cv2
import numpy as np
from pathlib import Path
from typing import List, Tuple, Optional
from tqdm import tqdm

# ==========================================
# 1. 环境与路径设置
# ==========================================
paths_to_add = [
    '/root',         
    '/root/task'   
]
for p in paths_to_add:
    if p not in sys.path:
        sys.path.append(p)

from ultralytics import YOLO
class YOLOConfig:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
class YOLOTrainer:
    def __init__(self, config):
        self.config = config
        self.model = YOLO(f"{config.model_type}.pt")
    
    def train(self):
        return self.model.train(
            data=f"{self.config.dataset_path}/../dataset.yaml",  # 指向DRIVE根目录的dataset.yaml
            epochs=self.config.epochs,
            patience=self.config.patience,
            batch=self.config.batch_size,
            imgsz=self.config.image_size,
            device=self.config.device,
            project=self.config.project_name,
            name=self.config.experiment_name,
            pretrained=self.config.pretrained_path,
            mosaic=self.config.mosaic,
            degrees=self.config.degrees,
            mixup=self.config.mixup,
            copy_paste=self.config.copy_paste,
            fliplr=self.config.fliplr,
            flipud=self.config.flipud,
            hsv_h=self.config.hsv_h,
            hsv_s=self.config.hsv_s,
            hsv_v=self.config.hsv_v,
            lr0=self.config.lr0,
            lrf=self.config.lrf,
            warmup_epochs=self.config.warmup_epochs,
            weight_decay=self.config.weight_decay,
            box=self.config.box_loss,
            cls=self.config.cls_loss,
            dfl=self.config.dfl_loss,
            iou=self.config.iou_threshold,
            max_det=self.config.max_detections,
            save_period=self.config.save_period,
            multi_scale=self.config.multi_scale,
            wandb_mode=self.config.wandb_mode
        )

# ==========================================
# 2. 优化版Mask转YOLO标签类 (解决血管框细碎/漏标问题)
# ==========================================
class MaskToYOLOConverter:
    """优化版：适配血管细长特征的标签生成器"""
    def __init__(self, class_id: int = 0):
        self.class_id = class_id
    
    def mask_to_bboxes(self, mask: np.ndarray, min_area: int = 20) -> List[Tuple[float, float, float, float]]:
        # 1. 二值化+降噪（关键：先清理小噪点，避免生成无效框）
        if mask.max() > 1:
            mask = (mask > 127).astype(np.uint8)
        else:
            mask = mask.astype(np.uint8)
        
        # 形态学闭运算：连接血管断点，减少细碎框
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        
        bboxes = []
        height, width = mask.shape
        
        # 优化网格策略：血管是细长的，用4x4网格（减少切割过度）
        grid_rows = 4
        grid_cols = 4
        step_h = height // grid_rows
        step_w = width // grid_cols
        
        for r in range(grid_rows):
            for c in range(grid_cols):
                y_start = r * step_h
                x_start = c * step_w
                y_end = (r + 1) * step_h if r < grid_rows - 1 else height
                x_end = (c + 1) * step_w if c < grid_cols - 1 else width
                
                patch = mask[y_start:y_end, x_start:x_end]
                if patch.sum() == 0:
                    continue
                
                # 找轮廓时保留更多细节（适配细长血管）
                contours, _ = cv2.findContours(patch, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
                
                for contour in contours:
                    lx, ly, lw, lh = cv2.boundingRect(contour)
                    if lw * lh < min_area:
                        continue
                    
                    # 坐标还原+归一化（保留6位小数，提升精度）
                    global_x = x_start + lx
                    global_y = y_start + ly
                    x_center = (global_x + lw / 2) / width
                    y_center = (global_y + lh / 2) / height
                    width_norm = lw / width
                    height_norm = lh / height
                    
                    # 过滤极窄/极高的异常框（避免标注噪声）
                    if width_norm < 0.005 or height_norm < 0.005:
                        continue
                    
                    bboxes.append((x_center, y_center, width_norm, height_norm))
        
        return bboxes

    def convert_single_mask(self, mask_path: Path, output_path: Path, min_area: int = 50):
        mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"⚠️ 无法读取mask文件: {mask_path}")
            return
        
        bboxes = self.mask_to_bboxes(mask, min_area)
        
        output_path.parent.mkdir(parents=True, exist_ok=True)
        with open(output_path, 'w') as f:
            for bbox in bboxes:
                x_center, y_center, width, height = bbox
                f.write(f"{self.class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
    
    def convert_dataset_inplace(self, base_path: str, min_area: int = 50, splits: List[str] = ['train', 'val']):
        base_path = Path(base_path)
        # 生成dataset.yaml（关键：补充数据集配置）
        self._generate_dataset_yaml(base_path)
        
        for split in splits:
            print(f"\n处理 {split} 数据集...")
            split_dir = base_path / split
            mask_dir = split_dir / 'mask' if (split_dir / 'mask').exists() else split_dir / 'masks'
            label_dir = split_dir / 'labels'
            
            # 清理旧标签+缓存
            if label_dir.exists():
                shutil.rmtree(label_dir)
            label_dir.mkdir(parents=True, exist_ok=True)
            
            cache_path = split_dir / 'labels.cache'
            if cache_path.exists():
                os.remove(cache_path)
                print(f"🗑️ 删除缓存: {cache_path}")
            
            if not mask_dir.exists():
                print(f"⚠️ 未找到mask目录: {mask_dir}")
                continue
            
            mask_files = list(mask_dir.glob('*.png')) + list(mask_dir.glob('*.jpg')) + list(mask_dir.glob('*.gif'))
            print(f"找到 {len(mask_files)} 个mask文件")
            
            for mask_file in tqdm(mask_files, desc=f"转换 {split} 标签"):
                raw_name = mask_file.stem.replace('_mask', '').replace('_manual1', '')
                label_file = label_dir / f"{raw_name}.txt"
                self.convert_single_mask(mask_file, label_file, min_area)
    
    def _generate_dataset_yaml(self, base_path: Path):
        """自动生成YOLO所需的dataset.yaml文件"""
        yaml_content = f"""
# DRIVE眼底血管数据集配置
path: {base_path.absolute()}  # 数据集根目录
train: train/images  # 训练集图片路径（需确保存在）
val: val/images      # 验证集图片路径（需确保存在）
test: 

# 类别
nc: 1  # 类别数
names: ['vessel']  # 类别名（替换原scar，匹配血管检测）

# 验证时的评估指标
metrics: ['precision', 'recall', 'mAP50', 'mAP50-95']
"""
        yaml_path = base_path / 'dataset.yaml'
        with open(yaml_path, 'w', encoding='utf-8') as f:
            f.write(yaml_content.strip())
        print(f"📝 生成数据集配置文件: {yaml_path}")

# ==========================================
# 3. 执行转换与训练（全优化参数）
# ==========================================

# 3.1 转换Mask到YOLO标签（优化后）
print("🚀 开始转换血管Mask到YOLO标签...")
converter = MaskToYOLOConverter(class_id=0)
converter.convert_dataset_inplace(
    base_path='../../../datasets/DRIVE',
    min_area=50,  # 过滤极小噪点，保留有效血管
    splits=['train', 'val']
)

# 3.2 配置训练参数（核心优化）
print("\n⚙️ 配置优化后的训练参数...")
config = YOLOConfig(
    # ========== 模型优化 ==========
    model_type="yolo11s",  # 升级为YOLO11s（比n版强3倍，显存占用可控）
    device='cuda',
    pretrained_path='yolo11s.pt',  # 使用官方预训练权重

    # ========== 数据集配置 ==========
    dataset_path='../../../datasets/DRIVE',  # 指向根目录（匹配dataset.yaml）
    val_dataset_path='../../../datasets/DRIVE',
    class_names=['vessel'],  # 修正类别名：scar→vessel（关键！）

    # ========== 训练核心参数 ==========
    epochs=100,
    patience=30,  # 延长早停耐心值，给模型更多收敛时间
    batch_size=16,
    image_size=640,
    multi_scale=True,  # 多尺度训练（适配不同大小的血管）

    # ========== 超参数优化（适配血管检测） ==========
    box_loss=10.0,     # 提升框损失权重，强化血管位置回归
    cls_loss=0.3,      # 降低分类损失（单类别无需高权重）
    dfl_loss=1.5,
    lr0=0.0005,        # 降低初始学习率，避免震荡
    lrf=0.01,          # 学习率衰减系数（原0.0001过低）
    warmup_epochs=5,   # 缩短预热轮次
    weight_decay=0.001,
    save_period=50,

    # ========== 数据增强（适配血管特征） ==========
    mosaic=0.3,        # 降低mosaic（避免血管拼接失真）
    mixup=0.0,         # 关闭mixup（细血管mixup后特征消失）
    copy_paste=0.0,    # 关闭copy_paste（避免血管重复标注）
    degrees=5.0,       # 降低旋转角度（避免血管扭曲）
    fliplr=0.2,        # 降低水平翻转（眼底血管左右结构固定）
    flipud=0.0,        # 关闭垂直翻转（无意义）
    hsv_h=0.05,        # 轻微色调增强
    hsv_s=0.1,         # 轻微饱和度增强
    hsv_v=0.2,         # 降低亮度增强（避免血管特征丢失）

    # ========== 检测参数 ==========
    iou_threshold=0.55,  # 提升IoU阈值（适配血管检测）
    conf_threshold=0.2,  # 训练时置信度阈值
    max_detections=500,  # 增加最大检测数（血管分支多）

    # ========== 项目配置 ==========
    project_name="runs",
    experiment_name="yolo_vessel_detection",  # 修正实验名（匹配血管检测）
    wandb_mode="disabled"
)

# 3.3 开始训练
print("\n🔥 开始训练YOLO血管检测模型...")
trainer = YOLOTrainer(config)
results = trainer.train()

print(f"\n✅ 训练完成！")
print(f"最佳权重保存在: runs/{config.experiment_name}/weights/best.pt")

🚀 开始转换血管Mask到YOLO标签...
📝 生成数据集配置文件: ../../../datasets/DRIVE/dataset.yaml

处理 train 数据集...
找到 100 个mask文件


转换 train 标签: 100%|██████████| 100/100 [00:00<00:00, 290.68it/s]



处理 val 数据集...
找到 20 个mask文件


转换 val 标签: 100%|██████████| 20/20 [00:00<00:00, 349.53it/s]



⚙️ 配置优化后的训练参数...

🔥 开始训练YOLO血管检测模型...


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [1]:
import sys
import os
import shutil
import cv2
import numpy as np
from pathlib import Path
from typing import List, Tuple, Optional
from tqdm import tqdm

# ==========================================
# 1. 环境与路径设置
# ==========================================

from ultralytics import YOLO
class YOLOConfig:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
class YOLOTrainer:
    def __init__(self, config):
        self.config = config
        self.model = YOLO(f"{config.model_type}.pt")

    def train(self):
        return self.model.train(
            data=f"{self.config.dataset_path}/dataset.yaml",
            epochs=self.config.epochs,
            patience=self.config.patience,
            batch=self.config.batch_size,
            imgsz=self.config.image_size,
            device=self.config.device,
            project=self.config.project_name,
            name=self.config.experiment_name,
            pretrained=self.config.pretrained_path,
            mosaic=self.config.mosaic,
            degrees=self.config.degrees,
            mixup=self.config.mixup,
            copy_paste=self.config.copy_paste,
            fliplr=self.config.fliplr,
            flipud=self.config.flipud,
            hsv_h=self.config.hsv_h,
            hsv_s=self.config.hsv_s,
            hsv_v=self.config.hsv_v,
            lr0=self.config.lr0,
            lrf=self.config.lrf,
            warmup_epochs=self.config.warmup_epochs,
            weight_decay=self.config.weight_decay,
            box=self.config.box_loss,
            cls=self.config.cls_loss,
            dfl=self.config.dfl_loss,
            iou=self.config.iou_threshold,
            max_det=self.config.max_detections,
            save_period=self.config.save_period,
            multi_scale=self.config.multi_scale
        )

# ==========================================
# 2. 优化版Mask转YOLO标签类 (适配label目录+21_training/21_manual1命名规则)
# ==========================================
class MaskToYOLOConverter:
    """优化版：适配血管细长特征 + label目录 + 21_training/21_manual1命名规则"""
    def __init__(self, class_id: int = 0):
        self.class_id = class_id

    def mask_to_bboxes(self, mask: np.ndarray, min_area: int = 20) -> List[Tuple[float, float, float, float]]:
        # 1. 二值化+降噪（关键：先清理小噪点，避免生成无效框）
        if mask.max() > 1:
            mask = (mask > 127).astype(np.uint8)
        else:
            mask = mask.astype(np.uint8)

        # 形态学闭运算：连接血管断点，减少细碎框
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        bboxes = []
        height, width = mask.shape

        # 优化网格策略：血管是细长的，用4x4网格（减少切割过度）
        grid_rows = 4
        grid_cols = 4
        step_h = height // grid_rows
        step_w = width // grid_cols

        for r in range(grid_rows):
            for c in range(grid_cols):
                y_start = r * step_h
                x_start = c * step_w
                y_end = (r + 1) * step_h if r < grid_rows - 1 else height
                x_end = (c + 1) * step_w if c < grid_cols - 1 else width

                patch = mask[y_start:y_end, x_start:x_end]
                if patch.sum() == 0:
                    continue

                # 找轮廓时保留更多细节（适配细长血管）
                contours, _ = cv2.findContours(patch, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

                for contour in contours:
                    lx, ly, lw, lh = cv2.boundingRect(contour)
                    if lw * lh < min_area:
                        continue

                    # 坐标还原+归一化（保留6位小数，提升精度）
                    global_x = x_start + lx
                    global_y = y_start + ly
                    x_center = (global_x + lw / 2) / width
                    y_center = (global_y + lh / 2) / height
                    width_norm = lw / width
                    height_norm = lh / height

                    # 过滤极窄/极高的异常框（避免标注噪声）
                    if width_norm < 0.005 or height_norm < 0.005:
                        continue

                    bboxes.append((x_center, y_center, width_norm, height_norm))

        return bboxes

    def convert_single_mask(self, mask_path: Path, output_path: Path, min_area: int = 50):
        # 兼容tif格式的mask读取（关键：原代码只处理png/jpg/gif，补充tif）
        mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"⚠️ 无法读取mask文件: {mask_path}")
            return

        bboxes = self.mask_to_bboxes(mask, min_area)

        output_path.parent.mkdir(parents=True, exist_ok=True)
        with open(output_path, 'w') as f:
            for bbox in bboxes:
                x_center, y_center, width, height = bbox
                f.write(f"{self.class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    def convert_dataset_inplace(self, base_path: str, min_area: int = 50, splits: List[str] = ['train', 'val']):
        base_path = Path(base_path)
        # 生成dataset.yaml（关键：补充数据集配置）
        self._generate_dataset_yaml(base_path)

        for split in splits:
            print(f"\n处理 {split} 数据集...")
            split_dir = base_path / split
            
            # ========== 核心修改1：固定查找label目录 ==========
            mask_dir = split_dir / 'label'
            if not mask_dir.exists():
                print(f"⚠️ 未找到mask目录: {mask_dir}")
                continue
            
            # ========== 核心修改2：输出标签目录（避免冲突） ==========
            label_dir = split_dir / 'yolo_labels'

            # 清理旧标签+缓存
            if label_dir.exists():
                shutil.rmtree(label_dir)
            label_dir.mkdir(parents=True, exist_ok=True)

            cache_path = split_dir / 'labels.cache'
            if cache_path.exists():
                os.remove(cache_path)
                print(f"🗑️ 删除缓存: {cache_path}")

            # ========== 核心修改3：兼容tif格式的mask文件 ==========
            mask_files = list(mask_dir.glob('*.png')) + list(mask_dir.glob('*.jpg')) + \
                         list(mask_dir.glob('*.gif')) + list(mask_dir.glob('*.tif'))
            print(f"找到 {len(mask_files)} 个mask文件 (路径: {mask_dir})")

            for mask_file in tqdm(mask_files, desc=f"转换 {split} 标签"):
                # ========== 核心修改4：适配21_manual1.tif → 21_training.tif的命名对齐 ==========
                # 步骤1：从mask文件名提取核心编号（如21_manual1.tif → 21）
                mask_stem = mask_file.stem  # 21_manual1
                core_id = mask_stem.split('_')[0]  # 提取前缀编号：21
                
                # 步骤2：拼接图像文件名（21 + _training.tif），确保标签名和图像名对齐
                # 最终标签名：21_training.txt（匹配图像21_training.tif）
                label_filename = f"{core_id}_training.txt"
                label_file = label_dir / label_filename
                
                self.convert_single_mask(mask_file, label_file, min_area)

    def _generate_dataset_yaml(self, base_path: Path):
        """自动生成YOLO所需的dataset.yaml文件（适配新的标签目录）"""
        yaml_content = f"""
# DRIVE眼底血管数据集配置
path: {base_path.absolute()}  # 数据集根目录
train: train/images  # 训练集图片路径（需确保存在，图像名如21_training.tif）
val: val/images      # 验证集图片路径
test:

# 类别
nc: 1  # 类别数
names: ['vessel']  # 类别名（血管）

# 标签路径（显式指定，避免YOLO查找错误）
train_labels: train/yolo_labels
val_labels: val/yolo_labels

# 验证时的评估指标
metrics: ['precision', 'recall', 'mAP50', 'mAP50-95']
"""
        yaml_path = base_path / 'dataset.yaml'
        with open(yaml_path, 'w', encoding='utf-8') as f:
            f.write(yaml_content.strip())
        print(f"📝 生成数据集配置文件: {yaml_path}")

# ==========================================
# 3. 执行转换与训练（全优化参数）
# ==========================================

# 3.1 转换Mask到YOLO标签（适配label目录+命名规则）
print("🚀 开始转换血管Mask到YOLO标签...")
converter = MaskToYOLOConverter(class_id=0)
converter.convert_dataset_inplace(
    base_path='../../../datasets/DRIVE',  # 替换为你的数据集根路径
    min_area=50,
    splits=['train', 'val']
)

# 3.2 配置训练参数（核心优化）
print("\n⚙️ 配置优化后的训练参数...")
config = YOLOConfig(
    # ========== 模型优化 ==========
    model_type="yolo11s",
    device='cuda',
    pretrained_path='yolo11s.pt',

    # ========== 数据集配置 ==========
    dataset_path='../../../datasets/DRIVE',  # 替换为你的数据集根路径
    val_dataset_path='../../../datasets/DRIVE',
    class_names=['vessel'],

    # ========== 训练核心参数 ==========
    epochs=100,
    patience=30,
    batch_size=16,
    image_size=640,
    multi_scale=True,

    # ========== 超参数优化（适配血管检测） ==========
    box_loss=10.0,
    cls_loss=0.3,
    dfl_loss=1.5,
    lr0=0.0005,
    lrf=0.01,
    warmup_epochs=5,
    weight_decay=0.001,
    save_period=50,

    # ========== 数据增强（适配血管特征） ==========
    mosaic=0.3,
    mixup=0.0,
    copy_paste=0.0,
    degrees=5.0,
    fliplr=0.2,
    flipud=0.0,
    hsv_h=0.05,
    hsv_s=0.1,
    hsv_v=0.2,

    # ========== 检测参数 ==========
    iou_threshold=0.55,
    conf_threshold=0.2,
    max_detections=500,

    # ========== 项目配置 ==========
    project_name="runs",
    experiment_name="yolo_vessel_detection",
    wandb="disabled"
)

# 3.3 开始训练
print("\n🔥 开始训练YOLO血管检测模型...")
trainer = YOLOTrainer(config)
results = trainer.train()

print(f"\n✅ 训练完成！")
print(f"最佳权重保存在: runs/{config.experiment_name}/weights/best.pt")

🚀 开始转换血管Mask到YOLO标签...
📝 生成数据集配置文件: ../../../datasets/DRIVE/dataset.yaml

处理 train 数据集...
🗑️ 删除缓存: ../../../datasets/DRIVE/train/labels.cache
找到 100 个mask文件 (路径: ../../../datasets/DRIVE/train/label)


转换 train 标签: 100%|██████████| 100/100 [00:00<00:00, 610.11it/s]



处理 val 数据集...
🗑️ 删除缓存: ../../../datasets/DRIVE/val/labels.cache
找到 20 个mask文件 (路径: ../../../datasets/DRIVE/val/label)


转换 val 标签: 100%|██████████| 20/20 [00:00<00:00, 260.02it/s]



⚙️ 配置优化后的训练参数...

🔥 开始训练YOLO血管检测模型...
Transferred 499/499 items from pretrained weights
Ultralytics 8.3.237 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32109MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=10.0, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.3, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../../../datasets/DRIVE/dataset.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.2, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.05, hsv_s=0.1, hsv_v=0.2, imgsz=640, int8=False, iou=0.55, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=500, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=0.3, multi_scale=True, name=yolo_vessel_detection3, nbs

In [2]:
from ultralytics import YOLO
model = YOLO("/root/task/src/YoloSAM/drive/runs/yolo_vessel_detection3/weights/best.pt")
# 对这张图做预测，保存最终检测结果
results = model.predict(
    source="/root/task/datasets/DRIVE/val/images/01_test.tif",  # 替换为这张图的路径
    save=True,  # 保存带检测框的图
    conf=0.5,   # 过滤低置信度框
    iou=0.5     # 去除重叠框
)
print("最终检测图保存至：", results[0].save_dir)


image 1/1 /root/task/datasets/DRIVE/val/images/01_test.tif: 640x640 16 vessels, 4.2ms
Speed: 1.7ms preprocess, 4.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /root/task/src/YoloSAM/drive/runs/detect/predict3
最终检测图保存至： /root/task/src/YoloSAM/drive/runs/detect/predict3
